In [7]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
data_train= pd.read_csv('../input/train_file.csv')

In [10]:
data_test=pd.read_csv('../input/test_file.csv')

In [11]:
data_train

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084
...,...,...,...,...,...,...,...,...,...,...,...
55927,jQ3CeLRCb9,Fidel Castro Lashes Out at Obama After Cuba Visit,Retired Cuban leader Fidel Castro slammed Pres...,Wall Street Journal,obama,2016-03-29 01:35:06,794,10,5,-0.135417,-0.055902
55928,akNYeJ8opY,JOHN CRISP | Obama's strategic reaction to Bru...,President Obama caught some predictable flak f...,Kitsap Sun,obama,2016-03-29 01:35:08,0,0,0,0.236228,0.056110
55929,n2DGs0c8IG,Think Trump's 45 Percent Tariffs Are Bad? Try ...,While Trump wants to put large tariffs on impo...,Huffington Post,obama,2016-03-29 01:35:09,102,4,0,0.025747,0.114820
55930,P0EBiaSEjq,Microsoft finally releases giant Surface,Microsoft’s business customers are finally beg...,TechEye,microsoft,2016-03-29 01:38:00,0,0,0,0.000000,-0.028296


In [12]:
data_train.drop(['PublishDate', 'Topic', 'Source','IDLink'], axis=1, inplace=True)

In [13]:
data_test.drop(['PublishDate', 'Topic', 'Source','IDLink'], axis=1, inplace=True)

In [14]:
import sklearn

In [15]:
data_train=pd.concat([data_train.drop(['Facebook', 'GooglePlus', 'LinkedIn'], axis=1),pd.DataFrame(sklearn.preprocessing.Normalizer().fit_transform(data_train.iloc[:,2:5]), columns=['Facebook', 'GooglePlus', 'LinkedIn'])], axis=1)
data_test=pd.concat([data_test.drop(['Facebook', 'GooglePlus', 'LinkedIn'], axis=1),pd.DataFrame(sklearn.preprocessing.Normalizer().fit_transform(data_test.iloc[:,2:5]), columns=['Facebook', 'GooglePlus', 'LinkedIn'])], axis=1)

In [16]:
#For Train Data
t=[]
h=[]
for i in data_train['Title']:
    letters_only = re.sub("[^a-zA-Z]"," ",i)
    t.append(letters_only)

for i in data_train['Headline']:
    letters_only = re.sub("[^a-zA-Z]"," ",i)
    h.append(letters_only)

    
data_train['Title']=t    
data_train['Headline']=h
 

data_train['Title'] = data_train['Title'].str.lower()
data_train['Headline'] = data_train['Headline'].str.lower()


data_train['Title'] = data_train['Title'].str.strip()
data_train['Headline'] = data_train['Headline'].str.strip()




# For test-Data

t1=[]
h1=[]
for i in data_test['Title']:
    letters_only = re.sub("[^a-zA-Z]"," ",i)
    t1.append(letters_only)

for i in data_test['Headline']:
    letters_only = re.sub("[^a-zA-Z]"," ",i)
    h1.append(letters_only)
    
    
data_test['Title']=t1    
data_test['Headline']=h1


data_test['Title'] = data_test['Title'].str.lower()
data_test['Headline'] = data_test['Headline'].str.lower()


data_test['Title'] = data_test['Title'].str.strip()
data_test['Headline'] = data_test['Headline'].str.strip()


In [17]:
del t
del t1
del h
del h1

In [18]:
data_train['Title'] = data_train['Title'].str.split()  
data_train['Headline'] = data_train['Headline'].str.split() 


data_test['Title'] = data_test['Title'].str.split()  
data_test['Headline'] = data_test['Headline'].str.split() 

In [19]:
stop = stopwords.words("english")
def removestopwords(y):   
    stopwordremoved =[w for w in y if not w in stop]
    return(" ".join(stopwordremoved))

In [20]:
#For Train
column_size = data_train.shape[0]
cleaned1 = []
cleaned2 = []
for i in range( 0, column_size):
    cleaned1.append(removestopwords(data_train['Title'][i]))
    cleaned2.append(removestopwords(data_train['Headline'][i]))
data_train['Title'] = cleaned1
data_train['Headline'] = cleaned2

#For Test
column_size = data_test.shape[0]
cleaned1 = []
cleaned2 = []
for i in range( 0, column_size):
    cleaned1.append(removestopwords(data_test['Title'][i]))
    cleaned2.append(removestopwords(data_test['Headline'][i]))
data_test['Title'] = cleaned1
data_test['Headline'] = cleaned2

In [21]:
data_train

,Title,Headline,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,obama lays wreath arlington national cemetery,obama lays wreath arlington national cemetery ...,0.000000,-0.053300,-0.577350,-0.577350,-0.577350
1,look health chinese economy,tim haywood investment director business unit ...,0.208333,-0.156386,-0.577350,-0.577350,-0.577350
2,nouriel roubini global economy back,nouriel roubini nyu professor chairman roubini...,-0.425210,0.139754,-0.577350,-0.577350,-0.577350
3,finland gdp expands q,finland economy expanded marginally three mont...,0.000000,0.026064,-0.577350,-0.577350,-0.577350
4,tourism govt spending buoys thai economy january,tourism public spending continued boost econom...,0.000000,0.141084,-0.577350,-0.577350,-0.577350
...,...,...,...,...,...,...,...
55927,fidel castro lashes obama cuba visit,retired cuban leader fidel castro slammed pres...,-0.135417,-0.055902,0.999901,0.012593,0.006297
55928,john crisp obama strategic reaction brussels,president obama caught predictable flak politi...,0.236228,0.056110,0.000000,0.000000,0.000000
55929,think trump percent tariffs bad try obama,trump wants put large tariffs imports major tr...,0.025747,0.114820,0.999232,0.039186,0.000000
55930,microsoft finally releases giant surface,microsoft business customers finally beginning...,0.000000,-0.028296,0.000000,0.000000,0.000000


In [22]:
data_test

,Title,Headline,Facebook,GooglePlus,LinkedIn
0,sliding economy fg fights back n trn tsa funds,budget passed national assembly n trillion war...,0.000000,0.000000,1.000000
1,microsoft shows hololens bring distant family ...,recent microsoft research video shows augmente...,0.994144,0.016432,0.106809
2,microsoft twitter robot praises hitler trump r...,microsoft teamed bing create taytweets account...,0.996546,0.083045,0.000000
3,flood central bank moves get world economy rut,central bankers managed steer world economy cl...,0.000000,0.000000,1.000000
4,usd jpy bears lining mixed u economy outlook,however streak seven day gains might end marke...,1.000000,0.000000,0.000000
...,...,...,...,...,...
37283,stocks rise investors key us economy ahead friday,june employment report viewed crucial gauge he...,-0.169031,0.507093,0.845154
37284,russian pm proposes use conservative tough sce...,addition establish stimulating economic policy...,-0.707107,0.000000,0.707107
37285,palestinian government uses foreign aid pay te...,palestinian government spends nearly million a...,0.980581,0.196116,0.000000
37286,palestine youth orchestra prepares first uk tour,palestine youth orchestra prepares first uk to...,0.000000,0.000000,0.000000


In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lemm = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return(" ".join(lemm)) 

In [25]:
#For Train
clean1=[]
clean2=[]
column_size = data_train.shape[0]
for i in range( 0, column_size):
    clean1.append(lemmatize_text(data_train['Title'][i]))
    clean2.append(lemmatize_text(data_train['Headline'][i]))
data_train['Title']=clean1
data_train['Headline']=clean2

#For Test
clean1=[]
clean2=[]
column_size = data_test.shape[0]
for i in range( 0, column_size):
    clean1.append(lemmatize_text(data_test['Title'][i]))
    clean2.append(lemmatize_text(data_test['Headline'][i]))
data_test['Title']=clean1
data_test['Headline']=clean2

In [26]:
porter_stemmer = PorterStemmer()
def stemmer(text):
    stem = [porter_stemmer.stem(w) for w in w_tokenizer.tokenize(text)]
    return(" ".join(stem)) 

In [27]:
#For Train
stem1=[]
stem2=[]
column_size = data_train.shape[0]
for i in range( 0, column_size):
    stem1.append(stemmer(data_train['Title'][i]))
    stem2.append(stemmer(data_train['Headline'][i]))
data_train['Title']=stem1
data_train['Headline']=stem2

#For Test
stem1=[]
stem2=[]
column_size = data_test.shape[0]
for i in range( 0, column_size):
    stem1.append(stemmer(data_test['Title'][i]))
    stem2.append(stemmer(data_test['Headline'][i]))
data_test['Title']=stem1
data_test['Headline']=stem2

In [28]:
del cleaned1
del cleaned2
del clean1
del clean2
del stem1
del stem2

In [29]:
data_train

,Title,Headline,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,obama lay wreath arlington nation cemeteri,obama lay wreath arlington nation cemeteri pre...,0.000000,-0.053300,-0.577350,-0.577350,-0.577350
1,look health chines economi,tim haywood invest director busi unit head fix...,0.208333,-0.156386,-0.577350,-0.577350,-0.577350
2,nouriel roubini global economi back,nouriel roubini nyu professor chairman roubini...,-0.425210,0.139754,-0.577350,-0.577350,-0.577350
3,finland gdp expand q,finland economi expand margin three month end ...,0.000000,0.026064,-0.577350,-0.577350,-0.577350
4,tourism govt spend buoy thai economi januari,tourism public spend continu boost economi jan...,0.000000,0.141084,-0.577350,-0.577350,-0.577350
...,...,...,...,...,...,...,...
55927,fidel castro lash obama cuba visit,retir cuban leader fidel castro slam presid ba...,-0.135417,-0.055902,0.999901,0.012593,0.006297
55928,john crisp obama strateg reaction brussel,presid obama caught predict flak polit right w...,0.236228,0.056110,0.000000,0.000000,0.000000
55929,think trump percent tariff bad tri obama,trump want put larg tariff import major trade ...,0.025747,0.114820,0.999232,0.039186,0.000000
55930,microsoft final releas giant surfac,microsoft busi custom final begin take deliver...,0.000000,-0.028296,0.000000,0.000000,0.000000


In [30]:
data_test

,Title,Headline,Facebook,GooglePlus,LinkedIn
0,slide economi fg fight back n trn tsa fund,budget pass nation assembl n trillion war ches...,0.000000,0.000000,1.000000
1,microsoft show hololen bring distant famili me...,recent microsoft research video show augment r...,0.994144,0.016432,0.106809
2,microsoft twitter robot prais hitler trump rec...,microsoft team bing creat taytweet account rob...,0.996546,0.083045,0.000000
3,flood central bank move get world economi rut,central banker manag steer world economi clear...,0.000000,0.000000,1.000000
4,usd jpi bear line mix u economi outlook,howev streak seven day gain might end market t...,1.000000,0.000000,0.000000
...,...,...,...,...,...
37283,stock rise investor key u economi ahead friday,june employ report view crucial gaug health ec...,-0.169031,0.507093,0.845154
37284,russian pm propos use conserv tough scenario,addit establish stimul econom polici budget co...,-0.707107,0.000000,0.707107
37285,palestinian govern us foreign aid pay terroris...,palestinian govern spend nearli million annual...,0.980581,0.196116,0.000000
37286,palestin youth orchestra prepar first uk tour,palestin youth orchestra prepar first uk tour ...,0.000000,0.000000,0.000000


In [31]:
vectorizer = TfidfVectorizer(max_features=5000)

In [32]:
xtrain=pd.concat([pd.DataFrame(vectorizer.fit_transform(data_train['Title']).toarray())  ,pd.DataFrame(vectorizer.fit_transform(data_train['Headline']).toarray())], axis=1)
data_train=data_train.drop(['Title', 'Headline'],axis=1)
xtrain=pd.concat([xtrain, data_train.drop(['SentimentTitle','SentimentHeadline'], axis=1)], axis=1)

In [33]:
xtest=pd.concat([pd.DataFrame(vectorizer.fit_transform(data_test['Title']).toarray())  ,pd.DataFrame(vectorizer.fit_transform(data_test['Headline']).toarray())], axis=1)
data_test=data_test.drop(['Title', 'Headline'],axis=1)
xtest=pd.concat([xtest, data_test], axis=1)

In [34]:
y=data_train.loc[:,'SentimentTitle':'SentimentHeadline']

In [35]:
y

,SentimentTitle,SentimentHeadline
0,0.000000,-0.053300
1,0.208333,-0.156386
2,-0.425210,0.139754
3,0.000000,0.026064
4,0.000000,0.141084
...,...,...
55927,-0.135417,-0.055902
55928,0.236228,0.056110
55929,0.025747,0.114820
55930,0.000000,-0.028296


In [36]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor 

In [38]:
#mo= MultiOutputRegressor(RandomForestRegressor(n_jobs=-1), n_jobs=-1)
mo=MLPRegressor(hidden_layer_sizes=(100, ), 
             activation='relu', solver='adam', 
             alpha=0.0001, batch_size='auto', 
             learning_rate='adaptive', 
             learning_rate_init=0.001, 
             power_t=0.5, max_iter=200, 
             shuffle=True, random_state=None, 
             tol=0.0001, verbose=False, 
             warm_start=False, momentum=0.9, 
             nesterovs_momentum=True, early_stopping=False,
             validation_fraction=0.1, beta_1=0.9,
             beta_2=0.999, epsilon=1e-08,
             n_iter_no_change=10, max_fun=15000)
mo.fit(xtrain,y)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='adaptive',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [40]:
mo.score(xtrain,y)

/opt/conda/lib/python3.6/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.9650157832902722

In [41]:
idx= pd.read_csv('../input/test_file.csv')
idx=idx.set_index('IDLink')

In [42]:
pred=mo.predict(xtest)
Rohit=pd.DataFrame(pred, columns=['SentimentTitle', 'SentimentHeadline'], index=idx.index)
Rohit.to_csv('/kaggle/working/Rohit.csv',index=False)

NameError: name 'files' is not defined